# Data analysis

## Analysis of each variable

All the variables will be analysed one by one, as some of them aren't representative of the problem and can be ommited.

In [1]:
training_set<-read.csv("../Data/TrainingSet/4910797b-ee55-40a7-8668-10efd5c1b960.csv",header=TRUE) # loads the training set csv file (it's magic)
training_labels<-read.csv("../Data/TrainingLabel/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv", header=TRUE) # Loads the corresponding labels
training_set <- merge(training_set, training_labels)
dim(training_set)
names(training_set)

[1] 59400    41

[1] "id"                    "amount_tsh"            "date_recorded"        
 [4] "funder"                "gps_height"            "installer"            
 [7] "longitude"             "latitude"              "wpt_name"             
[10] "num_private"           "basin"                 "subvillage"           
[13] "region"                "region_code"           "district_code"        
[16] "lga"                   "ward"                  "population"           
[19] "public_meeting"        "recorded_by"           "scheme_management"    
[22] "scheme_name"           "permit"                "construction_year"    
[25] "extraction_type"       "extraction_type_group" "extraction_type_class"
[28] "management"            "management_group"      "payment"              
[31] "payment_type"          "water_quality"         "quality_group"        
[34] "quantity"              "quantity_group"        "source"               
[37] "source_type"           "source_class"          "waterpoint_type"      
[40] "waterpoint_type_group" "status_group"

## List of categories for a column and count their occurence

In [3]:
count_categories <- function (data, column) {
    temp = data.frame(data["id"], data[column], data["status_group"])
    temp_names = as.data.frame(table(temp[column]))
    names(temp_names)[1] <- column
    return(temp_names)
}

count_categories(training_set, "scheme_management")

scheme_management,Freq
<fct>,<int>
,3877
Company,1061
None,1
Other,766
Parastatal,1680
Private operator,1063
SWC,97
Trust,72
VWC,36793


## Count of status_group (label) for each category

In [32]:
library(dplyr)

count_status <- function(data, column) {
    number_categories <- dim(unique(data[column]))[1]
    res_matrix <- matrix(,4, number_categories)  # Create matrix with 3 rows (labels) and N cols (number of categories)
        
    count <- 1
    for (category in unique(data[,column])) {
        filtered <- filter(data, data[,column] == category)
        temp_categories <- count_categories(filtered, "status_group")
        for (i in 1:3) {
            res_matrix[i, count] <- temp_categories$Freq[i]
        }
        res_matrix[4, count] <- res_matrix[1, count] / (res_matrix[1, count] + res_matrix[2, count] + res_matrix[3, count])
        count <- count + 1
    }
    res_matrix[is.na(res_matrix)] <- 0
    rownames(res_matrix) <- c("functional", "need repair", "non functional", "func %")
    colnames(res_matrix) <- unique(data[,column])
    return (res_matrix)
}

## Scheme management variable

In [33]:
# only print 2 digits
options(digits=2)

count_status(training_set, "scheme_management")

,VWC,,Water Board,Private operator,WUA,Water authority,WUG,Other,Company,Parastatal,SWC,Trust,None
functional,1.9e+04,1873.00,2053.00,729.00,1995.00,1618.00,3006.00,458.0,534.0,966.00,20.00,46.00,1
need repair,2.3e+03,223.00,111.00,23.00,239.00,448.00,672.00,22.0,37.0,202.00,1.00,5.00,0
non functional,1.5e+04,1781.00,584.00,311.00,649.00,1087.00,1528.00,286.0,490.0,512.00,76.00,21.00,0
func %,5.2e-01,0.48,0.75,0.69,0.69,0.51,0.58,0.6,0.5,0.57,0.21,0.64,0


Scheme management is rather uncorrellated with the status group.

In [34]:
count_status(training_set, "scheme_name")

,,Mgun,Losaa-Kia water supply,Lwih,Ipal,Mapinduzi,Mkalamo water supply,Mradi wa maji wa kilagano,Mlimani pipe line,Msin,⋯,TM Lawate water supply,Nyashimo,Villagers,Charles Mushi Branch line,Tina,Maga,BRA,Nampemba,Mradi wa maji wa kibilizi,Kima
functional,1.4e+04,5,145,7,1,9,4,23.00,16,5,⋯,1,1,1,1,1,1,1,1,1,1
need repair,2.0e+03,3,7,0,9,0,18,2.00,9,3,⋯,0,0,0,0,0,0,0,0,0,0
non functional,1.2e+04,0,0,0,0,0,0,10.00,0,0,⋯,0,0,0,0,0,0,0,0,0,0
func %,5.1e-01,0,0,0,0,0,0,0.66,0,0,⋯,0,0,0,0,0,0,0,0,0,0


Scheme name has too much NA values and different categories. This cannot be used by our model

In [35]:
count_status(training_set, "permit")

,True,False,
functional,2.2e+04,9045.00,1673.00
need repair,2.7e+03,1320.00,300.00
non functional,1.5e+04,7127.00,1083.00
func %,5.5e-01,0.52,0.55


Permit is lightly correllated with the status group. This measure need to be kept and encoded into an ont-hot encoded matrix. How do we treat N/A values?

In [36]:
count_status(training_set, "construction_year")

,0,2008,1999,2007,2010,1984,2012,1981,2002,1977,⋯,1970,2013,1965,1968,1963,1971,1967,1961,1962,1966
functional,1.1e+04,1839.0,694.00,1081.00,2023.00,365.00,726.00,85.00,692.00,62.00,⋯,99.00,102.00,11.00,22.00,31.00,82.00,31.00,2.000,7.00,6
need repair,1.8e+03,127.0,47.00,116.00,90.00,66.00,51.00,11.00,58.00,10.00,⋯,15.00,1.00,1.00,4.00,6.00,6.00,17.00,3.000,1.00,11
non functional,8.4e+03,647.0,238.00,390.00,532.00,348.00,307.00,142.00,325.00,130.00,⋯,297.00,73.00,7.00,51.00,48.00,57.00,40.00,16.000,22.00,0
func %,5.1e-01,0.7,0.71,0.68,0.76,0.47,0.67,0.36,0.64,0.31,⋯,0.24,0.58,0.58,0.29,0.36,0.57,0.35,0.095,0.23,0


Construction year is an important measure, need to be kept and encoded as an integer (by the pump age). Many NA values exists and should be dealt with.

In [41]:
count_status(training_set, "extraction_type")

,afridev,other - rope pump,mono,gravity,nira/tanira,submersible,other,swn 80,windmill,india mark ii,ksb,cemo,india mark iii,other - swn 81,climax,other - play pump,walimi,other - mkulima/shinyanga
functional,1200.00,293.00,1082.00,16048.0,5421.00,2626.00,1029.00,2090.00,50.00,1448.0,703.0,45.0,44.00,120.00,8,29.00,23.00,2
need repair,42.00,17.00,129.00,2701.0,641.00,227.00,206.00,212.00,7.00,79.0,26.0,9.0,1.00,7.00,24,1.00,12.00,0
non functional,528.00,141.00,1654.00,8031.0,2092.00,1911.00,5195.00,1368.00,60.00,873.0,686.0,36.0,53.00,102.00,0,55.00,13.00,0
func %,0.68,0.65,0.38,0.6,0.66,0.55,0.16,0.57,0.43,0.6,0.5,0.5,0.45,0.52,0,0.34,0.48,0


 Extraction type also looks like an importan measure, as it influences the operating mode of the pump. Some classes have better results than others. This should also be encoded into a one-hot matrix (but has many dimensions)

In [42]:
count_status(training_set, "extraction_type_group")

,afridev,rope pump,mono,gravity,nira/tanira,submersible,other,swn 80,wind-powered,india mark ii,other motorpump,india mark iii,other handpump
functional,1200.00,293.00,1082.00,16048.0,5421.00,3329.00,1029.00,2090.00,50.00,1448.0,53.00,44.00,172.00
need repair,42.00,17.00,129.00,2701.0,641.00,253.00,206.00,212.00,7.00,79.0,9.00,1.00,20.00
non functional,528.00,141.00,1654.00,8031.0,2092.00,2597.00,5195.00,1368.00,60.00,873.0,60.00,53.00,172.00
func %,0.68,0.65,0.38,0.6,0.66,0.54,0.16,0.57,0.43,0.6,0.43,0.45,0.47


Perhaps it is better to use the group than the extraction type, as it has less categories but still shows differences between the categories.

In [43]:
count_status(training_set, "extraction_type_class")

,handpump,rope pump,motorpump,gravity,submersible,other,wind-powered
functional,1.0e+04,293.00,1135.00,16048.0,3329.00,1029.00,50.00
need repair,1.0e+03,17.00,138.00,2701.0,253.00,206.00,7.00
non functional,5.1e+03,141.00,1714.00,8031.0,2597.00,5195.00,60.00
func %,6.3e-01,0.65,0.38,0.6,0.54,0.16,0.43


Using a type class also has again less categories than the group and shows differences between the categorieds.

Ultimately only one of those three categories should be used.

In [44]:
count_status(training_set, "management")

,vwc,water board,private operator,wua,wug,company,water authority,other,unknown,parastatal,trust,other - school
functional,20425.0,2170.00,1476.00,1751.00,3906.0,267.00,446.00,505.0,224.0,1020.00,46.00,23.00
need repair,2791.0,265.00,44.00,205.00,645.0,15.00,52.00,55.0,27.0,211.00,6.00,1.00
non functional,17291.0,498.00,451.00,579.00,1964.0,403.00,406.00,284.0,310.0,537.00,26.00,75.00
func %,0.5,0.74,0.75,0.69,0.6,0.39,0.49,0.6,0.4,0.58,0.59,0.23


Management is also an important class, maybe management group is better?

In [46]:
count_status(training_set, "management_group")

,user-group,commercial,other,unknown,parastatal
functional,2.8e+04,2235.00,528.00,224.0,1020.00
need repair,3.9e+03,117.00,56.00,27.0,211.00
non functional,2.0e+04,1286.00,359.00,310.0,537.00
func %,5.4e-01,0.61,0.56,0.4,0.58


Management group is less corellated than management

In [47]:
count_status(training_set, "payment")

,unknown,never pay,pay per bucket,pay when scheme fails,pay monthly,pay annually,other
functional,3528.00,1.1e+04,6090.00,2429.00,5482.00,2740.00,611.00
need repair,432.00,1.9e+03,409.00,277.00,927.00,247.00,118.00
non functional,4197.00,1.2e+04,2486.00,1208.00,1891.00,655.00,325.00
func %,0.43,4.5e-01,0.68,0.62,0.66,0.75,0.58


Payment looks like an important value, (can maybe be encoded into an integer?)

In [48]:
count_status(training_set, "payment_type")

,unknown,never pay,per bucket,on failure,monthly,annually,other
functional,3528.00,1.1e+04,6090.00,2429.00,5482.00,2740.00,611.00
need repair,432.00,1.9e+03,409.00,277.00,927.00,247.00,118.00
non functional,4197.00,1.2e+04,2486.00,1208.00,1891.00,655.00,325.00
func %,0.43,4.5e-01,0.68,0.62,0.66,0.75,0.58


Duplicate column from payment, can be discarded

In [49]:
count_status(training_set, "water_quality")

,milky,soft,salty,salty abandoned,coloured,unknown,fluoride,fluoride abandoned
functional,438.00,2.9e+04,2220.00,174.00,246.0,264.00,151.00,6
need repair,14.00,3.9e+03,225.00,72.00,54.0,35.00,13.00,11
non functional,352.00,1.8e+04,2411.00,93.00,190.0,1577.00,36.00,0
func %,0.54,5.7e-01,0.46,0.51,0.5,0.14,0.76,0


Might be an interesting data but must be processed (maybe encoded into an integer again?)

In [50]:
count_status(training_set, "quality_group")

,milky,good,salty,colored,unknown,fluoride
functional,438.00,2.9e+04,2394.00,246.0,264.00,157.00
need repair,14.00,3.9e+03,297.00,54.0,35.00,13.00
non functional,352.00,1.8e+04,2504.00,190.0,1577.00,47.00
func %,0.54,5.7e-01,0.46,0.5,0.14,0.72


Better than water_quality, should be used an encoded as integer.

In [51]:
count_status(training_set, "quantity")

,enough,insufficient,dry,seasonal,unknown
functional,2.2e+04,7916.00,1.6e+02,2325.00,213.00
need repair,2.4e+03,1450.00,3.7e+01,416.00,14.00
non functional,9.1e+03,5763.00,6.1e+03,1309.00,562.00
func %,6.5e-01,0.52,2.5e-02,0.57,0.27


Important, should be kept.

In [53]:
count_status(training_set, "quantity_group")

,enough,insufficient,dry,seasonal,unknown
functional,2.2e+04,7916.00,1.6e+02,2325.00,213.00
need repair,2.4e+03,1450.00,3.7e+01,416.00,14.00
non functional,9.1e+03,5763.00,6.1e+03,1309.00,562.00
func %,6.5e-01,0.52,2.5e-02,0.57,0.27


duplicate, can be dropped

In [56]:
count_status(training_set, "source")

,shallow well,machine dbh,spring,rainwater harvesting,river,lake,other,dam,hand dtw,unknown
functional,8324.00,5422.00,1.1e+04,1386.0,5465.00,162.00,126.00,253.00,497.00,32.00
need repair,957.00,491.00,1.3e+03,314.0,1221.00,12.00,1.00,24.00,17.00,4.00
non functional,7543.00,5162.00,5.2e+03,595.0,2926.00,591.00,85.00,379.00,360.00,30.00
func %,0.49,0.49,6.2e-01,0.6,0.57,0.21,0.59,0.39,0.57,0.48


In [57]:
Important, should be kept

ERROR: Error in parse(text = x, srcfile = src): <text>:1:10: ',' inattendu(e)
1: Important,
             ^


In [58]:
count_status(training_set, "source_type")

,shallow well,borehole,spring,rainwater harvesting,river/lake,other,dam
functional,8324.00,5919.0,1.1e+04,1386.0,5627.00,158.00,253.00
need repair,957.00,508.0,1.3e+03,314.0,1233.00,5.00,24.00
non functional,7543.00,5522.0,5.2e+03,595.0,3517.00,115.00,379.00
func %,0.49,0.5,6.2e-01,0.6,0.54,0.57,0.39


Perhaps better than previous.

In [59]:
count_status(training_set, "source_class")

,groundwater,surface,unknown
functional,2.5e+04,7266.00,158.00
need repair,2.7e+03,1571.00,5.00
non functional,1.8e+04,4491.00,115.00
func %,5.4e-01,0.55,0.57


Not much correlation, should be dropped.

In [60]:
count_status(training_set, "waterpoint_type")

,hand pump,communal standpipe multiple,communal standpipe,other,improved spring,cattle trough,dam
functional,1.1e+04,2235.00,1.8e+04,840.00,563.00,84.00,6
need repair,1.0e+03,648.00,2.3e+03,293.00,85.00,2.00,1
non functional,5.7e+03,3220.00,8.5e+03,5247.00,136.00,30.00,0
func %,6.2e-01,0.37,6.2e-01,0.13,0.72,0.72,0


High correlation, should be kept

In [61]:
count_status(training_set, "waterpoint_type_group")

,hand pump,communal standpipe,other,improved spring,cattle trough,dam
functional,1.1e+04,2.0e+04,840.00,563.00,84.00,6
need repair,1.0e+03,2.9e+03,293.00,85.00,2.00,1
non functional,5.7e+03,1.2e+04,5247.00,136.00,30.00,0
func %,6.2e-01,5.8e-01,0.13,0.72,0.72,0


Again a more compact still interesting feature

# To summerize the different features
* Scheme management is uncorrelated
* Scheme name have too much unique names and NA values, not interesting
* Permit is important (impact on reparation) and should be one-hot encoded
* Construction year looks important and should be changed to age
* Extraction type, extraction type group and extraction type class are important, one of them three should be kept
* Management and management group are imporant, one of both should be kept
* payment is important, maybe can be encoded as integer
* payment type is DUPLICATE
* water quality and quantity are also important, one of both should be kept, maybe can be encoded as integer
* water quantity is important, one should be kept, maybe encoded as integer
* water quantity group is DUPLICATE.
* source and source type, one should be kept. One hot encoded
* source class is reather uncorellated. Should be dropped
* waterpoint type and waterpoint type are importnat, one should be kept.